<a href="https://colab.research.google.com/github/HyeRim-Choi/imgAnimation/blob/main/imgAnimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **라이브러리 설치**

In [ ]:
# Python 에서 YAML 파일을 파싱하거나, 반대로 Python 객체를 YAML 파일로 내보낼 때 사용하는 라이브러리
!pip install PyYAML==5.3.1

In [ ]:
# 이미지를 처리하기 위한 파이썬 모듈
!pip install imageio-ffmpeg

In [ ]:
# 동영상을 처리하기 위한 파이썬 모듈
!pip install ffmpeg

# **코드 불러오기 및 경로 설정**

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

In [ ]:
cd first-order-model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# **이미지 resize**

In [ ]:
from PIL import Image
image1 = Image.open('/content/gdrive/MyDrive/first-order-motion-model/yu_01.png')
# image1 = Image.open('/content/gdrive/MyDrive/first-order-motion-model/mona.jpg')
# image1 = Image.open('/content/gdrive/MyDrive/first-order-motion-model/jico_01.png')
imag1_size = image1.size
print(imag1_size)

# 이미지 resize
image1 = image1.resize((280,280)) 
image1.show()
imag1_size = image1.size
print(imag1_size)
image1.save("../image.png",'PNG')

# **FFmpeg을 사용하여 mp4 영상을 crop**

In [ ]:
!ffmpeg -i /content/gdrive/My\ Drive/first-order-motion-model/iusu.mp4 -ss 00:00:0.0 -t 00:00:26.28 -filter:v "crop=280:280:500:70" -async 1 video.mp4

# **영상에서 오디오 추출**

In [ ]:
!ffmpeg -i video.mp4 audio.mp3

# **입력영상과 사진 불러오기**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

# 입력 이미지
source_image = imageio.imread('../image.png')
# 입력 영상
reader = imageio.get_reader('video.mp4')


# 이미지와 영상 256x256로 resize
source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
HTML(display(source_image, driving_video).to_html5_video())

# **영상 만들기**

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

# 완성된 영상 저장
imageio.mimsave('../finishedVideo.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

HTML(display(source_image, driving_video, predictions).to_html5_video())

# **완성된 mp4영상과 추출한 mp3 파일 합치기**

In [ ]:
!ffmpeg -i '../finishedVideo.mp4' -i audio.mp3 -c copy finalVer.mp4